# Import

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchtext import data
from torchtext import datasets

import pandas as pd
import numpy as np
import time
import jsonlines

In [2]:
%load_ext autoreload
%autoreload 2

from seewhence import models
from seewhence import train

In [3]:
# Directories & Paths
OUTPUT_DIR = '/home/ben/data/acl_2020/output/seewhence/'
reddit_path = '/home/ben/data/acl_2020/reddit.jsonl'
twitter_path = '/home/ben/data/acl_2020/twitter.jsonl'

In [4]:
# Directories & Paths
OUTPUT_DIR = 'data'

# Load and split Sarcasm Data

In [5]:
# reddit_path = '/home/ben/data/acl_2020/reddit.jsonl'
# twitter_path = '/home/ben/data/acl_2020/twitter.jsonl'
# train_split = 0.8
# %run ./preprocessing.ipynb

# Torch Text Dataset

In [6]:
# Fields
RESPONSE = data.Field()
LABEL = data.LabelField(dtype = torch.float)
fields = {'response': ('text', RESPONSE), 'label': ('label', LABEL)}

In [7]:
train_data, valid_data, test_data = data.TabularDataset.splits(
                                        path = OUTPUT_DIR,
                                        train = 'train.csv',
                                        validation ='valid.csv',
                                        test = 'test.csv',
                                        format = 'csv',
                                        fields = fields
)

## Build Vocab

In [8]:
MAX_VOCAB_SIZE = 25_000

RESPONSE.build_vocab(train_data, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors = "glove.twitter.27B.200d", 
                 unk_init = torch.Tensor.normal_)

In [9]:
LABEL.build_vocab(train_data)

# Iterator

In [10]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

BATCH_SIZE = 50

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    sort_key=lambda x: len(x.text),
    sort_within_batch = True,
    device = device)

# Define Model

In [11]:
INPUT_DIM = len(RESPONSE.vocab)
EMBEDDING_DIM = 200
OUTPUT_DIM = 1
N_EPOCHS = 5
PAD_IDX = RESPONSE.vocab.stoi[RESPONSE.pad_token]
output_path = 'models_nn_'

model = models.FastText(INPUT_DIM, 
                        EMBEDDING_DIM, 
                        OUTPUT_DIM, 
                        PAD_IDX)

In [12]:
pretrained_embeddings = RESPONSE.vocab.vectors
model.embedding.weight.data.copy_(pretrained_embeddings)

UNK_IDX = RESPONSE.vocab.stoi[RESPONSE.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

In [13]:
optimizer = optim.Adam(model.parameters())
criterion = nn.BCEWithLogitsLoss()

In [14]:
model = model.to(device)
criterion = criterion.to(device)

In [15]:
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train.train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = train.evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = train.epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        if output_path:
            torch.save(model.state_dict(), output_path)
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 1s
	Train Loss: 0.682 | Train Acc: 55.85%
	 Val. Loss: 0.667 |  Val. Acc: 64.00%


In [16]:
model.load_state_dict(torch.load(output_path))
test_loss, test_acc = train.evaluate(model, test_iterator, criterion)
print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.667 | Test Acc: 61.74%
